* uniform random initialization
* C-shunted qubit : profile aping

In [1]:
from multiprocessing import Pool
from DiSuQ.Torch.optimization import uniformParameters,initializationParallelism
from DiSuQ.Torch import models
from DiSuQ.Torch.optimization import OrderingOptimization,PolynomialOptimization
from DiSuQ.Torch.optimization import loss_Transition
from torch import tensor, float32 as float
from numpy import arange,linspace
from DiSuQ.utils import plotCompare
import pickle
from torch import set_num_threads
set_num_threads(35)

In [2]:
with open('../Examples/target_fluxqubit.p', 'rb') as content: target_info = pickle.load(content)
target_spectrum = target_info['spectrum']
E10 = target_spectrum[:,1] - target_spectrum[:,0]
E20 = target_spectrum[:,2] - target_spectrum[:,0]
target = {'E10':E10[[0,20,-1]],'E20':E20[[0,20,-1]]}

In [3]:
basis = {'O':[4],'I':[8,8],'J':[]}; rep = 'K'
circuit = models.shuntedQubit(basis,sparse=False)

In [4]:
flux_range = linspace(0,1,3,endpoint=True)

In [5]:
flux_profile = [{'I':flux} for flux in flux_range]

In [6]:
lossObjective = loss_Transition(tensor(target['E10'],dtype=float),tensor(target['E20'],dtype=float))

In [7]:
optim = OrderingOptimization(circuit,representation=rep)

In [19]:
parameters = uniformParameters(circuit,3)

In [10]:
with Pool(20) as multi:
    Search = multi.map(initializationParallelism(optim,lossObjective,flux_profile),parameters)

AttributeError: Can't pickle local object 'initializationParallelism.<locals>.optimization'

### optimization analysis

In [ ]:
for dLogs,dParams,dCircuit in Search:
    